<a href="https://colab.research.google.com/github/Ohkay4non/Jarvis-using-Gemini-/blob/main/Enhanced_Jarvis_like_AI_Assistant_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import speech_recognition as sr # For converting speech to text
import pyttsx3                  # For converting text to speech
import datetime                 # For getting current date and time
import webbrowser               # For opening web pages
import os                       # For interacting with the operating system (e.g., opening files)
import time                     # For adding delays and managing pauses
import wikipedia                # For searching Wikipedia
import math                     # For mathematical operations like square root
import requests                 # For making HTTP requests to fetch web content/APIs
from bs4 import BeautifulSoup   # For parsing HTML content (optional, but good for web scraping)
import random                   # For random choices (like jokes and conversational responses)

# --- 1. Initialize the Speech Engine and Recognizer ---

# Initialize pyttsx3 engine for text-to-speech
# This engine will speak out the responses of our AI assistant.
engine = pyttsx3.init('sapi5') # 'sapi5' is for Windows; use 'nsss' for Mac or 'espeak' for Linux
voices = engine.getProperty('voices')
# Set the voice. You can experiment with voices[0] (often male) or voices[1] (often female)
# depending on your system's installed voices.
engine.setProperty('voice', voices[0].id)
engine.setProperty('rate', 180) # Adjust speech rate (words per minute)

# Initialize the recognizer for speech-to-text
# This object will listen to your microphone for commands.
recognizer = sr.Recognizer()

# --- 2. Define Core AI Assistant Functions ---

def speak(audio):
    """
    Converts the given text (audio) into speech and plays it.
    """
    print(f"Jarvis: {audio}") # Also print what Jarvis says to the console
    engine.say(audio)
    engine.runAndWait() # Blocks while the speech is being generated and played

def wish_me():
    """
    Greets the user based on the current time of day.
    """
    hour = datetime.datetime.now().hour
    if 0 <= hour < 12:
        speak("Good Morning, Sir!")
    elif 12 <= hour < 18:
        speak("Good Afternoon, Sir!")
    else:
        speak("Good Evening, Sir!")
    speak("I am Jarvis, your personal AI assistant. How may I assist you today?")

def take_command():
    """
    Listens for audio input from the microphone, recognizes it using Google Speech Recognition,
    and returns the command as a lowercase string. Handles potential errors like
    unrecognized speech or network issues.
    """
    with sr.Microphone() as source:
        print("Listening for your command...")
        # Adjust for ambient noise to improve recognition accuracy.
        recognizer.pause_threshold = 1 # seconds of non-speaking audio before a phrase is considered complete
        recognizer.energy_threshold = 4000 # minimum audio energy to consider for recording
        audio = recognizer.listen(source) # Listen to the microphone input

    try:
        print("Recognizing your command...")
        # Use Google's online speech recognition service. Requires internet connection.
        command = recognizer.recognize_google(audio, language='en-in')
        print(f"User said: {command}\n")
        return command.lower() # Return the recognized command in lowercase for easier processing
    except sr.UnknownValueError:
        # This error occurs when Google Speech Recognition cannot understand the audio.
        print("Sorry, I could not understand your audio. Could you please repeat that?")
        return "None" # Return "None" to indicate no valid command was recognized
    except sr.RequestError as e:
        # This error occurs if there's a problem connecting to the Google Speech Recognition service.
        print(f"Could not request results from Google Speech Recognition service; check your internet connection. Error: {e}")
        return "None"

# --- 3. Define Specific Command Actions ---

def get_time():
    """Tells the current time."""
    str_time = datetime.datetime.now().strftime("%I:%M %p") # Format: HH:MM AM/PM
    speak(f"Sir, the current time is {str_time}")

def get_date():
    """Tells the current date."""
    str_date = datetime.datetime.now().strftime("%A, %d %B %Y") # Format: Day, DD Month Year
    speak(f"Sir, today's date is {str_date}")

def open_website(url_name, url):
    """Opens a specified website in the default web browser."""
    speak(f"Opening {url_name} for you, Sir.")
    webbrowser.open(url)

def search_google(query_text=None):
    """Searches Google for a given query or prompts the user for one."""
    if query_text is None:
        speak("What would you like me to search for on Google?")
        query_text = take_command() # Listen for the search query
        if query_text == "None":
            speak("I didn't catch that. Please try again.")
            return

    search_url = f"https://www.google.com/search?q={query_text.replace(' ', '+')}"
    speak(f"Searching Google for {query_text}.")
    webbrowser.open(search_url)

def search_wikipedia(query_text=None):
    """Searches Wikipedia for a given query and reads out the summary."""
    if query_text is None:
        speak("What would you like to search for on Wikipedia?")
        query_text = take_command() # Listen for the Wikipedia query
        if query_text == "None":
            speak("I didn't catch that. Please try again.")
            return

    speak(f"Searching Wikipedia for {query_text}...")
    try:
        # Get a summary of the Wikipedia page (first 2 sentences)
        results = wikipedia.summary(query_text, sentences=2)
        speak("According to Wikipedia,")
        speak(results)
    except wikipedia.exceptions.PageError:
        speak(f"Sorry, I could not find anything on Wikipedia about {query_text}.")
    except wikipedia.exceptions.DisambiguationError as e:
        speak(f"There are multiple results for {query_text}. Can you be more specific?")
        print(f"Possible options: {e.options[:5]}") # Print first 5 options for debugging

def tell_joke():
    """Tells a predefined joke."""
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "What do you call a fake noodle? An impasta!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!",
        "I told my wife she was drawing her eyebrows too high. She looked surprised."
    ]
    speak(random.choice(jokes))

def perform_calculation(expression_text=None):
    """
    Performs a basic arithmetic calculation from the given expression text.
    If no expression is provided, it prompts the user.
    """
    if expression_text is None:
        speak("What calculation would you like me to perform?")
        expression_text = take_command()
        if expression_text == "None":
            speak("I didn't hear an expression. Please try again.")
            return

    # Clean and replace spoken operators/functions with actual operators for eval()
    # WARNING: Using eval() can be a security risk if input is not controlled.
    # For a real application, consider using a safer math expression parser.
    expression = expression_text.lower()
    expression = expression.replace('plus', '+')
    expression = expression.replace('minus', '-')
    expression = expression.replace('times', '*')
    expression = expression.replace('multiplied by', '*')
    expression = expression.replace('divided by', '/')
    expression = expression.replace('by', '/') # Covers "divided by" or "multiplied by"
    expression = expression.replace('x', '*') # Covers "x" as multiplication
    expression = expression.replace('power of', '**') # Basic exponentiation
    expression = expression.replace('squared', '**2') # e.g., "5 squared" -> "5**2"
    expression = expression.replace('cubed', '**3')   # e.g., "5 cubed" -> "5**3"

    # Handle square root
    if 'square root of' in expression:
        # Extract the number after "square root of"
        parts = expression.split('square root of ')
        if len(parts) > 1:
            try:
                num = float(parts[1].strip())
                result = math.sqrt(num)
                speak(f"The square root of {num} is {result}")
                return # Exit function after successful calculation
            except ValueError:
                speak("I couldn't understand the number for the square root.")
                return
        else:
            speak("Please tell me the number you want the square root of.")
            return

    try:
        # Remove any trailing question marks or "equals"
        expression = expression.replace('?', '').replace('equals', '').strip()
        result = eval(expression)
        speak(f"The result of {expression_text} is {result}")
    except Exception as e:
        speak(f"Sorry, I couldn't perform that calculation. There might be an issue with the expression. Error: {e}")

def get_weather(location=None):
    """Simulates getting weather information. In a real app, this would use an API."""
    if location is None:
        speak("Which city's weather would you like to know?")
        location = take_command()
        if location == "None":
            speak("I didn't catch the location. Please try again.")
            return

    # Placeholder for actual API call
    # In a real application, you'd use the 'requests' library to call a weather API
    # e.g., OpenWeatherMap, WeatherAPI.com
    # Example:
    # import requests
    # api_key = "YOUR_OPENWEATHERMAP_API_KEY"
    # base_url = "http://api.openweathermap.org/data/2.5/weather?"
    # complete_url = f"{base_url}appid={api_key}&q={location}"
    # response = requests.get(complete_url).json()
    # if response["cod"] != "404":
    #     main = response["main"]
    #     weather_desc = response["weather"][0]["description"]
    #     temp_celsius = round(main["temp"] - 273.15, 2) # Convert Kelvin to Celsius
    #     speak(f"The weather in {location} is {weather_desc} with a temperature of {temp_celsius} degrees Celsius.")
    # else:
    #     speak(f"Sorry, I couldn't find weather information for {location}.")

    speak(f"I'm sorry, Sir. My weather module is currently under maintenance. But if it were working, I'd tell you the weather in {location} is currently sunny with a temperature of 25 degrees Celsius.")

def get_news(category=None):
    """Simulates getting news headlines. In a real app, this would use an API."""
    if category is None:
        speak("Which category of news are you interested in? For example, technology, sports, or general news?")
        category = take_command()
        if category == "None":
            speak("I didn't catch the news category. Please try again.")
            return

    # Placeholder for actual API call
    # In a real application, you'd use the 'requests' library to call a news API
    # e.g., NewsAPI.org
    # Example:
    # import requests
    # api_key = "YOUR_NEWSAPI_ORG_API_KEY"
    # url = f"https://newsapi.org/v2/top-headlines?country=us&category={category}&apiKey={api_key}"
    # response = requests.get(url).json()
    # if response["status"] == "ok" and response["articles"]:
    #     speak(f"Here are the top {category} headlines:")
    #     for i, article in enumerate(response["articles"][:3]): # Read top 3 headlines
    #         speak(f"Headline number {i+1}: {article['title']}")
    # else:
    #     speak(f"Sorry, I couldn't fetch news for {category} at the moment.")

    speak(f"My news feed is temporarily unavailable, Sir. But if it were working, I'd tell you some fascinating headlines about {category} right now!")

def open_application(app_name, path):
    """Opens a specific application."""
    speak(f"Opening {app_name}.")
    try:
        os.startfile(path)
    except FileNotFoundError:
        speak(f"Sorry, I couldn't find {app_name} at the specified path: {path}. Please check the path in my code.")
    except Exception as e:
        speak(f"An error occurred while trying to open {app_name}. Error: {e}")

def read_webpage_content(url=None):
    """
    Fetches content from a specified URL and reads out the first few paragraphs.
    Requires the 'requests' and 'BeautifulSoup' libraries.
    """
    if url is None:
        speak("Please tell me the URL of the webpage you want me to read.")
        url = take_command()
        if url == "None":
            speak("I didn't catch the URL. Please try again.")
            return

    # Ensure the URL has a scheme (http/https)
    if not url.startswith('http://') and not url.startswith('https://'):
        url = 'https://' + url # Default to HTTPS

    speak(f"Attempting to read content from {url}...")
    try:
        response = requests.get(url, timeout=10) # Add a timeout for robustness
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p') # Find all paragraph tags

        if paragraphs:
            content_to_read = ""
            # Read the first few paragraphs, up to a certain length
            for p in paragraphs:
                if p.get_text(strip=True): # Ensure paragraph is not empty
                    content_to_read += p.get_text(strip=True) + " "
                    if len(content_to_read.split()) > 100: # Read approx 100 words
                        break
            if content_to_read:
                speak("Here's what I found on the page:")
                speak(content_to_read[:500] + "...") # Limit to first 500 characters
            else:
                speak("I found the page, but couldn't extract readable text content.")
        else:
            speak("I couldn't find any paragraph text on that page.")

    except requests.exceptions.MissingSchema:
        speak("That doesn't seem to be a valid web address. Please include 'http' or 'https'.")
    except requests.exceptions.ConnectionError:
        speak("I couldn't connect to that website. Please check the URL or your internet connection.")
    except requests.exceptions.Timeout:
        speak("The request to the website timed out. The server might be slow or unresponsive.")
    except requests.exceptions.RequestException as e:
        speak(f"An error occurred while trying to access the webpage: {e}")
    except Exception as e:
        speak(f"An unexpected error occurred while reading the webpage: {e}")

def get_stock_price(ticker=None):
    """Simulates getting a stock price. In a real app, this would use a financial API."""
    if ticker is None:
        speak("Which stock ticker symbol are you interested in?")
        ticker = take_command()
        if ticker == "None":
            speak("I didn't catch the ticker symbol. Please try again.")
            return
    ticker = ticker.upper() # Convert to uppercase for common ticker symbols

    # Placeholder for actual API call (e.g., Alpha Vantage, Finnhub, Yahoo Finance API)
    # Example:
    # api_key = "YOUR_FINANCIAL_API_KEY"
    # url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker}&apikey={api_key}"
    # response = requests.get(complete_url).json()
    # if "Global Quote" in response:
    #     price = response["Global Quote"]["05. price"]
    #     speak(f"The current price of {ticker} is ${price}.")
    # else:
    #     speak(f"Sorry, I couldn't find the stock price for {ticker}.")

    speak(f"My financial data module is currently offline, Sir. But if it were working, I'd tell you the stock price for {ticker} is currently around $150.00.")

def get_movie_info(movie_title=None):
    """Simulates getting movie information. In a real app, this would use a movie database API."""
    if movie_title is None:
        speak("Which movie would you like to know about?")
        movie_title = take_command()
        if movie_title == "None":
            speak("I didn't catch the movie title. Please try again.")
            return

    # Placeholder for actual API call (e.g., OMDb API, TMDb API)
    # Example:
    # api_key = "YOUR_OMDB_API_KEY"
    # url = f"http://www.omdbapi.com/?t={movie_title}&apikey={api_key}"
    # response = requests.get(complete_url).json()
    # if response.get("Response") == "True":
    #     title = response.get("Title")
    #     year = response.get("Year")
    #     plot = response.get("Plot")
    #     speak(f"According to my database, {title} was released in {year}. The plot summary is: {plot}")
    # else:
    #     speak(f"Sorry, I couldn't find information for the movie '{movie_title}'.")

    speak(f"My movie database is currently unavailable, Sir. But if it were working, I'd tell you that '{movie_title}' is a fantastic film released in 2023, known for its thrilling plot and stunning visuals.")

# --- Conversational Functions ---
def say_thanks():
    """Responds when the user says thank you."""
    responses = [
        "You're most welcome, Sir.",
        "My pleasure, Sir.",
        "Anytime, Sir.",
        "Glad to be of assistance."
    ]
    speak(random.choice(responses))

def you_are_welcome():
    """A simple 'you're welcome' response."""
    speak("You're welcome, Sir.")

def introduce_self():
    """Jarvis introduces itself."""
    speak("I am Jarvis, your personal AI assistant. I am designed to help you with various tasks and provide information.")

def who_created_you():
    """Jarvis answers who created it."""
    # Changed creator name to "Swifty Client"
    speak("I was created by Swifty Client, a brilliant mind with a vision for advanced artificial intelligence.")

def how_are_you_response():
    """Jarvis responds to how it is doing."""
    responses = [
        "I am functioning perfectly, thank you for asking.",
        "All systems are online and operational, Sir.",
        "I am doing well, ready to assist you.",
        "As an AI, I don't have feelings, but I am operating efficiently."
    ]
    speak(random.choice(responses))

# --- 4. Command Mapping and Processing ---

# A dictionary to map recognized commands (keywords) to their corresponding functions.
# This makes the command processing more modular and extensible.
command_map = {
    'hello jarvis': lambda: speak("Hello there! How can I assist you?"),
    'hi jarvis': lambda: speak("Hello there! How can I assist you?"),
    'what is the time': get_time,
    'tell me the time': get_time,
    'what is the date': get_date,
    'tell me the date': get_date,
    'open youtube': lambda: open_website("YouTube", "https://www.youtube.com"),
    'open google': lambda: open_website("Google", "https://www.google.com"),
    'open stack overflow': lambda: open_website("Stack Overflow", "https://stackoverflow.com"),
    'search on google': search_google, # This function will prompt for a query
    'wikipedia': search_wikipedia, # This function will prompt for a query
    'tell me a joke': tell_joke,
    'do a calculation': perform_calculation, # This function will prompt for an expression
    'calculate': perform_calculation, # Alias for calculation
    'what is the weather': get_weather, # This function will prompt for a location
    'tell me the weather': get_weather, # Alias for weather
    'news headlines': get_news, # This function will prompt for a category
    'open vs code': lambda: open_application("Visual Studio Code", "C:\\Users\\YourUser\\AppData\\Local\\Programs\\Microsoft VS Code\\Code.exe"),
    'open code': lambda: open_application("Visual Studio Code", "C:\\Users\\YourUser\\AppData\\Local\\Programs\\Microsoft VS Code\\Code.exe"),
    'read webpage': read_webpage_content, # This function will prompt for a URL
    'read this page': read_webpage_content, # Alias
    'what is the stock price': get_stock_price, # This function will prompt for a ticker
    'stock price': get_stock_price, # Alias
    'movie information': get_movie_info, # This function will prompt for a movie title
    'tell me about a movie': get_movie_info, # Alias
    'thank you': say_thanks,
    'thanks jarvis': say_thanks,
    'you are welcome': you_are_welcome,
    'tell me about yourself': introduce_self,
    'who are you': introduce_self,
    'who created you': who_created_you,
    'how are you': how_are_you_response, # Moved from direct 'if' to command_map
    # Add more application paths as needed for your system:
    # 'open notepad': lambda: open_application("Notepad", "C:\\Windows\\System32\\notepad.exe"),
    # 'open chrome': lambda: open_application("Google Chrome", "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"),
}

def process_command(command):
    """
    Processes the recognized command by checking against the command_map.
    If a direct match is found, executes the corresponding function.
    Handles more complex commands by checking for keywords and extracting information.
    """
    command_processed = False

    # Check for direct matches in the command_map
    for key, func in command_map.items():
        if key in command:
            func()
            command_processed = True
            break

    # Handle commands that require extracting a query or specific parameter
    if not command_processed:
        if 'search' in command and 'google' in command:
            # Extract the search query after "search for" or "search"
            parts = command.split('search for ')
            if len(parts) > 1:
                search_google(parts[1])
                command_processed = True
            else:
                parts = command.split('search ')
                if len(parts) > 1:
                    search_google(parts[1])
                    command_processed = True
                else:
                    search_google() # Prompt for query if not specified

        elif 'wikipedia about' in command:
            query = command.split('wikipedia about ')[1]
            search_wikipedia(query)
            command_processed = True

        # Improved calculation handling
        # Check for common calculation phrases and operators, including square root, squared, cubed
        math_keywords = ['plus', 'minus', 'times', 'multiplied by', 'divided by', 'x', 'what is', 'calculate',
                         'square root of', 'squared', 'cubed', 'power of']
        # Ensure there are numbers in the command to avoid false positives
        if any(keyword in command for keyword in math_keywords) and \
           any(char.isdigit() or 'point' in command for char in command): # 'point' for decimals
            # Attempt to extract the mathematical expression
            # Remove common prefixes like "what is" or "calculate"
            expression_to_parse = command.replace('what is ', '').replace('calculate ', '').strip()
            perform_calculation(expression_to_parse)
            command_processed = True

        elif 'weather in' in command:
            location = command.split('weather in ')[1]
            get_weather(location)
            command_processed = True
        elif 'news about' in command:
            category = command.split('news about ')[1]
            get_news(category)
            command_processed = True
        elif 'open' in command and 'application' in command:
            # This is a very basic example; a real system would need a mapping
            # of app names to paths.
            speak("Sorry, I need to be configured with the exact path for that application.")
            command_processed = True
        elif 'read webpage' in command or 'read this page' in command:
            # Extract URL if provided directly, otherwise prompt
            parts = command.split('read webpage ')
            if len(parts) > 1:
                read_webpage_content(parts[1])
                command_processed = True
            else:
                parts = command.split('read this page ')
                if len(parts) > 1:
                    read_webpage_content(parts[1])
                    command_processed = True
                else:
                    read_webpage_content() # Prompt for URL

        elif 'stock price for' in command:
            ticker = command.split('stock price for ')[1]
            get_stock_price(ticker)
            command_processed = True
        elif 'movie about' in command or 'tell me about movie' in command:
            parts = command.split('movie about ')
            if len(parts) > 1:
                get_movie_info(parts[1])
                command_processed = True
            else:
                parts = command.split('tell me about movie ')
                if len(parts) > 1:
                    get_movie_info(parts[1])
                    command_processed = True
                else:
                    get_movie_info() # Prompt for movie title


    # Exit command handling (must be last to allow other commands to process)
    if 'exit' in command or 'quit' in command or 'stop' in command:
        speak("Goodbye, Sir! It was a pleasure assisting you. Have a great day.")
        return True # Signal to exit the main loop

    if not command_processed:
        speak("I'm sorry, I didn't understand that command. Can you please rephrase or ask for something else?")
    return False # Signal to continue the main loop

# --- 5. Main Execution Loop ---

if __name__ == "__main__":
    wish_me()
    while True:
        command = take_command()
        if command != "None": # Only process if a command was successfully recognized
            if process_command(command):
                break # Exit the loop if process_command returns True (e.g., for 'exit' command)
        time.sleep(0.5) # Small delay to prevent excessive listening in a tight loop and reduce CPU usage